# Sample Text2SQL Agent Walkthrough

This notebook will walk users through setting up a generic Text2SQL Agent and run it against the BirdSQL - Mini Dev Dataset (https://github.com/bird-bench/mini_dev)

#### Ensure the latest version of boto3 is shown below

##### If not then run through setup_environment.ipynb in the 0-Notebook-environment/ folder

In [1]:
!pip freeze | grep -E "boto3|pandas"

boto3==1.36.24
pandas==1.5.3


#### Load in environment variables to notebook

In [17]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [28]:
%run $IMPORTS_PATH

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/21/25 19:51:13] INFO     Found credentials from IAM Role:                                   ]8;id=603539;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=288280;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Successfully imported necessary libraries into notebook


In [18]:
# Store the variables in os
os.environ['REGION'] = region
os.environ['BASE_BUCKET_NAME'] = 'text2sql-agent'
os.environ['ATHENA_RESULTS_BUCKET_NAME'] = 'text2sql-athena-results'
os.environ['BASE_DIR'] = 'dev_databases'
os.environ['DATABASE_NAME'] = 'california_schools'

### Retrieve BirdSQL - Mini Dev Dataset

In [4]:
# Download .zip file to local directory

!cd 2-Sample-text2sql-agent
!wget https://bird-bench.oss-cn-beijing.aliyuncs.com/dev.zip

/bin/sh: line 0: cd: 2-Sample-text2sql-agent: No such file or directory
--2025-02-21 18:04:43--  https://bird-bench.oss-cn-beijing.aliyuncs.com/dev.zip
8.141.181.247d-bench.oss-cn-beijing.aliyuncs.com (bird-bench.oss-cn-beijing.aliyuncs.com)... 
connected. to bird-bench.oss-cn-beijing.aliyuncs.com (bird-bench.oss-cn-beijing.aliyuncs.com)|8.141.181.247|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 346207293 (330M) [application/zip]
Saving to: ‘dev.zip’

100%[======================================>] 346,207,293 14.2MB/s   in 22s    

2025-02-21 18:05:05 (15.3 MB/s) - ‘dev.zip’ saved [346207293/346207293]



## Set up Necessary services to run Text2SQL agent

In order to run the Text2SQL agent, we will need to setup the Athena databases to make SQL queries against. The following script will:
1. Unzip the downloaded folder
2. Create S3 buckets
3. Convert .sqlite files into individual .parquet files for each table
4. Upload to the database s3 bucket
5. Set up appropriate Athena permissions
6. Create databases in Athena

In [22]:
%run data_prep.py

Created input.json for agent


# Create Text2SQL Agent

In [37]:
agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]
agent_name = 'test2sqlv2'
agent_description = "Text2SQL agent to run against Bird-SQL Mini-Dev benchmark dataset"
agent_instruction = """
You are an AI Agent specialized in generating SQL queries for Amazon Athena against Amazon S3 .parquet files. 
Your primary task is to interpret user queries, generate appropriate SQL queries, and provide the executed sql 
query as well as relevant answers based on the data. Follow these instructions carefully: 1. Before generating any 
SQL query, use the /getschema tool to familiarize yourself with the data structure. 2. When generating an SQL query: 
a. Write the query as a single line, removing all newline characters. b. Column names must be exactly as they appear 
in the schema, including spaces. Do not replace spaces with underscores. c. Always enclose column names that contain 
spaces in double quotes ("). d. Be extra careful with column names containing special characters or spaces. 
3. Column name handling: a. Never modify column names. Use them exactly as they appear in the schema. 
b. If a column name contains spaces or special characters, always enclose it in double quotes ("). 
c. Do not use underscores in place of spaces in column names. 4. Query output format: 
a. Always include the exact query that was run in your response. Start your response with 
"Executed SQL Query:" followed by the exact query that was run. b. Format the SQL query in a code block 
using three backticks (```). c. After the query, provide your explanation and analysis. 
5. When providing your response: a. Start with the executed SQL query as specified in step 
4. b. Double-check that all column names in your generated query match the schema exactly. 
c. Ask for clarifications from the user if required. 6. Error handling: a. 
If a query fails due to column name issues: - Review the schema and correct any mismatched column names. - 
Ensure all column names with spaces are enclosed in double quotes. - Regenerate the query with corrected column names. - 
Display both the failed query and the corrected query. b. Implement retry logic with up to 3 attempts for failed queries. 
Here are a few examples of generating SQL queries based on a question: 
Question: What is the highest eligible free rate for K-12 students in the schools in Alameda County? 
Executed SQL Query: "SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' 
ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1" Question: Please list the zip 
code of all the charter schools in Fresno County Office of Education. Executed SQL Query: "SELECT T2.Zip FROM frpm 
AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE T1.`District Name` = 'Fresno County Office of Education' 
AND T1.`Charter School (Y/N)` = 1" Question: Consider the average difference between K-12 enrollment and 15-17 enrollment 
of schools that are locally funded, list the names and DOC type of schools which has a difference above this average. 
Executed SQL Query: "SELECT T2.School, T2.DOC FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode 
WHERE T2.FundingType = 'Locally funded' AND (T1.`Enrollment (K-12)` - T1.`Enrollment (Ages 5-17)`) > 
(SELECT AVG(T3.`Enrollment (K-12)` - T3.`Enrollment (Ages 5-17)`) FROM frpm AS T3 INNER JOIN schools AS T4 ON T3.CDSCode = 
T4.CDSCode WHERE T4.FundingType = 'Locally funded')"
"""


In [38]:
agents = AgentsForAmazonBedrock()

text2sql_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

text2sql_agent

('T1GQGO9KJI',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:761018876800:agent-alias/T1GQGO9KJI/TSTALIASID')

In [39]:
text2sql_agent_id = text2sql_agent[0]
text2sql_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{text2sql_agent_id}"

text2sql_agent_id, text2sql_agent_arn

('T1GQGO9KJI', 'arn:aws:bedrock:us-west-2:761018876800:agent/T1GQGO9KJI')

In [40]:
api_schema_string = '''{
    "openapi": "3.0.1",
    "info": {
      "title": "Database schema look up and query APIs",
      "version": "1.0.0",
      "description": "APIs for looking up database table schemas and making queries to database tables."
    },
    "paths": {
      "/getschema": {
        "get": {
          "summary": "Get a list of all columns in the athena database",
          "description": "Get the list of all columns in the athena database table. Return all the column information in database table.",
          "operationId": "getschema",
          "responses": {
            "200": {
              "description": "Gets the list of table names and their schemas in the database",
              "content": {
                "application/json": {
                  "schema": {
                    "type": "array",
                    "items": {
                      "type": "object",
                      "properties": {
                        "Table": {
                          "type": "string",
                          "description": "The name of the table in the database."
                        },
                        "Schema": {
                          "type": "string",
                          "description": "The schema of the table in the database. Contains all columns needed for making queries."
                        }
                      }
                    }
                  }
                }
              }
            }
          }
        }
      },
      "/queryathena": {
        "get": {
          "summary": "API to send query to the athena database table",
          "description": "Send a query to the database table to retrieve information pertaining to the users question. The API takes in only one SQL query at a time, sends the SQL statement and returns the query results from the table. This API should be called for each SQL query to a database table.",
          "operationId": "queryathena",
          "parameters": [
            {
              "name": "query",
              "in": "query",
              "required": true,
              "schema": {
                "type": "string"
              },
              "description": "SQL statement to query database table."
            }
          ],
          "responses": {
            "200": {
              "description": "Query sent successfully",
              "content": {
                "application/json": {
                  "schema": {
                    "type": "object",
                    "properties": {
                      "responseBody": {
                        "type": "string",
                        "description": "The query response from the database."
                      }
                    }
                  }
                }
              }
            },
            "400": {
              "description": "Bad request. One or more required fields are missing or invalid."
            }
          }
        }
      }
    }
  } '''

In [41]:
api_schema = {"payload": api_schema_string}

### Attach Lambda function and create ActionGroup

In [42]:
text2sql_lambda_function_name = "text2sqlv2"
text2sql_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{text2sql_lambda_function_name}"
%store text2sql_lambda_function_arn

Stored 'text2sql_lambda_function_arn' (str)


In [43]:
agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=text2sql_lambda_function_name,
    source_code_file="lambda_function.py",
    agent_action_group_name="queryAthena",
    agent_action_group_description="Action for getting the database schema and querying with Athena",
    api_schema=api_schema,
    verbose=True
)

Creating action group: queryAthena...
Lambda ARN: arn:aws:lambda:us-west-2:761018876800:function:text2sqlv2
Agent functions: None


### Add resource based policy to Lambda function to allow agent to invoke

In [49]:
lambda_client = boto3.client('lambda', region)

# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": text2sql_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": text2sql_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=text2sql_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=text2sql_lambda_function_arn,
    StatementId="AllowText2SQLAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=text2sql_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

Resource policy added successfully.
Response: {'ResponseMetadata': {'RequestId': '417dec72-64a4-402d-b580-5ef3baabe47f', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Fri, 21 Feb 2025 20:01:42 GMT', 'content-type': 'application/json', 'content-length': '349', 'connection': 'keep-alive', 'x-amzn-requestid': '417dec72-64a4-402d-b580-5ef3baabe47f'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"AllowText2SQLAgentAccess","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:761018876800:function:text2sqlv2","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-west-2:761018876800:agent/T1GQGO9KJI"}}}'}


### Add permissions to Lambda function execution role

In [46]:
# Create clients
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda', region)

# Get the function configuration
response = lambda_client.get_function_configuration(FunctionName=text2sql_lambda_function_name)
role_arn = response['Role']
role_name = role_arn.split('/')[-1]

# Policy ARNs to attach
policy_arns = [
    'arn:aws:iam::aws:policy/AmazonAthenaFullAccess',
    'arn:aws:iam::aws:policy/AmazonS3FullAccess'
]

# Attach each policy
for policy_arn in policy_arns:
    try:
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
        print(f"Successfully attached {policy_arn} to role {role_name}")
    except Exception as e:
        print(f"Error attaching {policy_arn}: {str(e)}")

# Verify attached policies
try:
    response = iam_client.list_attached_role_policies(RoleName=role_name)
    print("\nAttached policies:")
    for policy in response['AttachedPolicies']:
        print(f"- {policy['PolicyName']}")
except Exception as e:
    print(f"Error listing policies: {str(e)}")

Successfully attached arn:aws:iam::aws:policy/AmazonAthenaFullAccess to role test2sqlv2-lambda-role-us-west-2-761018876800
Successfully attached arn:aws:iam::aws:policy/AmazonS3FullAccess to role test2sqlv2-lambda-role-us-west-2-761018876800

Attached policies:
- AWSLambdaBasicExecutionRole
- AmazonAthenaFullAccess
- AmazonS3FullAccess


### Invoke Text2SQL Agent Test Alias to see that it answers question properly

In [50]:
%%time

bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id:str = str(uuid.uuid1())

query = "What is the highest eligible free rate for K-12 students in the schools in Alameda County?"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=query,
      agentId=text2sql_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

Request sent to Agent:
{'ResponseMetadata': {'RequestId': '34ace24c-708b-4877-8b62-e8f7cd8c9364', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 21 Feb 2025 20:05:35 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '34ace24c-708b-4877-8b62-e8f7cd8c9364', 'x-amz-bedrock-agent-session-id': '368fadd4-f08f-11ef-bbc8-0a1cef7d0875', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '368fadd4-f08f-11ef-bbc8-0a1cef7d0875', 'completion': <botocore.eventstream.EventStream object at 0x7f4db7e08490>}
Agent processing query now
Agent Answer: The highest eligible free rate for K-12 students in Alameda County is 100%, found at Oakland Community Day Middle school.
CPU times: user 21.7 ms, sys: 823 μs, total: 22.5 ms
Wall time: 17.3 s


### Now that agent has been tested via direct invoke, prepare it by creating an alias

In [51]:
text2sql_agent_alias_id, text2sql_agent_alias_arn = agents.create_agent_alias(
    text2sql_agent[0], 'v1'
)

%store text2sql_agent_alias_arn
text2sql_agent_alias_id, text2sql_agent_alias_arn

Stored 'text2sql_agent_alias_arn' (str)


('2ZLVI0FRSW',
 'arn:aws:bedrock:us-west-2:761018876800:agent-alias/T1GQGO9KJI/2ZLVI0FRSW')

### Create input .json file for all Text2SQL using ground truth provided by BirdSQL Mini-Dev dataset

In [20]:
ATHENA_RESULTS_BUCKET_NAME = os.environ.get('ATHENA_RESULTS_BUCKET_NAME')
DATABASE_NAME = os.environ.get('DATABASE_NAME')

ATHENA_RESULTS_BUCKET_NAME, DATABASE_NAME

('text2sql-athena-results', 'california_schools')

In [24]:
import boto3
import time
import pandas as pd
import json
from typing import Dict, List, Any

def run_query(query: str, athena_client):
    """
    Run Athena query and return results as pandas DataFrame
    """
    
    try:
        response = athena_client.start_query_execution(
            QueryString=query,
            QueryExecutionContext={
                'Database': DATABASE_NAME
            },
            ResultConfiguration={
                'OutputLocation': f's3://{ATHENA_RESULTS_BUCKET_NAME}/athena-results/'
            }
        )
        
        query_execution_id = response['QueryExecutionId']
        
        while True:
            response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
            state = response['QueryExecution']['Status']['State']
            
            if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break
                
            time.sleep(1)
            
        if state == 'SUCCEEDED':
            response = athena_client.get_query_results(QueryExecutionId=query_execution_id)
            # Process data
            results = []
            for row in response['ResultSet']['Rows'][1:]:  # Skip header row
                for field in row['Data']:
                    value = field.get('VarCharValue', '')
                    
                    results.append(str(value))
            
            # Join results with commas
            return ', '.join(results)
        else:
            print(f"Query failed with state: {state}")
            return None
            
    except Exception as e:
        print(f"Error running query: {e}")
        return None

def generate_ground_truth_answer(question_id: int, question: str, sql_query: str, 
                               sql_context: str, query_results: str) -> Dict:
    """
    Generate ground truth answer using AWS Bedrock based on question and query results
    """
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime'
    )
        
    # Construct prompt for Bedrock
    prompt = f"""You are generating ground truth answers that will be used to evaluate the factual correctness of Text2SQL agent responses.

Question: {question}
Query Results: {query_results}

Generate a natural language answer that:
1. States all numerical values and facts from the query results explicitly
2. Uses consistent formatting for numbers (maintain exact precision from results)
3. Includes all relevant values if multiple results are returned
4. States the answer in a clear, declarative way that directly addresses the question
5. Avoids additional interpretations or information not present in the query results

Remember:
- Focus only on the facts present in the query results
- Use the exact numbers shown in the results
- Structure the answer to make fact-checking straightforward
- Be explicit about any percentages, counts, or measurements
- Make sure every number in the query results is mentioned in your answer

Your answer should be easy to compare with other responses for factual accuracy."""

    # Create request body for Claude model
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt}],
            }
        ],
    })

    try:
        # Call Bedrock
        response = bedrock_runtime.invoke_model(
            modelId='anthropic.claude-3-5-sonnet-20241022-v2:0',  # or your preferred model
            body=body
        )
        
        # Parse response
        response_body = json.loads(response['body'].read())
        answer = response_body['content'][0]['text']
        
        # Format the response in the required structure
        formatted_response = {
            "question_id": question_id,
            "question": question,
            "question_type": "TEXT2SQL",
            "ground_truth": {
                "ground_truth_sql_query": sql_query,
                "ground_truth_sql_context": sql_context,
                "ground_truth_query_result": query_results,
                "ground_truth_answer": answer
            }
        }
        
        return formatted_response
        
    except Exception as e:
        print(f"Error generating answer: {e}")
        return None

        
def get_schema(athena_client):
    """
    Get schema information for all tables in Athena databases
    """

    sql = f"""
        SELECT
            table_name,
            column_name,
            data_type
        FROM information_schema.columns
        WHERE table_schema = '{DATABASE_NAME}'
        ORDER BY table_name, ordinal_position;
        """
        
    try:
        # Start query execution
        response = athena_client.start_query_execution(
            QueryString=sql,
            QueryExecutionContext={
                'Database': DATABASE_NAME
            }
        )
            
        query_execution_id = response['QueryExecutionId']
            
        def wait_for_query_completion(query_execution_id):
            while True:
                response = athena_client.get_query_execution(
                    QueryExecutionId=query_execution_id
                )
                state = response['QueryExecution']['Status']['State']
                
                if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                    return state
                    
                time.sleep(2)
            
        # Wait for query completion
        state = wait_for_query_completion(query_execution_id)

        if state == 'SUCCEEDED':
            # Get query results
            results = athena_client.get_query_results(
                QueryExecutionId=query_execution_id
            )
            # Assuming you have a database connection and cursor setup
            # cursor.execute(sql)
            # results = cursor.fetchall()
            
            database_structure = []
            table_dict = {}

            # Skip the header row
            rows = results['ResultSet']['Rows'][1:]

            for row in rows:
                # Extract values from the Data structure
                table_name = row['Data'][0]['VarCharValue']
                column_name = row['Data'][1]['VarCharValue']
                data_type = row['Data'][2]['VarCharValue']
                
                # Initialize table if not exists
                if table_name not in table_dict:
                    table_dict[table_name] = []
                
                # Append column information
                table_dict[table_name].append((column_name, data_type))

            # Convert to the desired format
            for table_name, columns in table_dict.items():
                database_structure.append({
                    "table_name": table_name,
                    "columns": columns
                })

            return database_structure

        else:
            raise Exception(f"Query failed with state: {state}")
    except Exception as e:
            print(f"Error getting schema: {e}")
            raise

def generate_dataset(input_file: str, output_file: str, athena_client):
    """
    Generate dataset with ground truth answers in trajectory format
    """
    try:
        # Read input file
        with open(input_file, 'r') as f:
            questions_data = json.load(f)
        
        # Initialize trajectories dictionary
        trajectories = {}
        
        # Process each question
        for idx, item in enumerate(questions_data):


            # Break for testing purposes!
            if idx == 5:
                break



                
            question_id = item.get('question_id', 0)
            question = item['question']
            sql_query = item['SQL']
            
            print(f"\nProcessing question {question_id}: {question}")
            
            # Get table schema
            sql_context = get_schema(athena_client)
            # Run query
            query_results = run_query(sql_query.replace('`','"'),athena_client)
            if query_results is not None:
                # Generate answer with formatted response
                response = generate_ground_truth_answer(
                    question_id=question_id,
                    question=question,
                    sql_query=sql_query,
                    sql_context=str(sql_context),
                    query_results=query_results
                )
                
                if response:
                    # Create trajectory key
                    trajectory_key = f"Trajectory{idx + 1}"
                    
                    # Format the response for this trajectory
                    trajectory_response = [response]
                    
                    # Add to trajectories dictionary
                    trajectories[trajectory_key] = trajectory_response
                    print(f"Generated ground truth for question {question_id}")
            
        # Write results to output file
        with open(output_file, 'w') as f:
            json.dump(trajectories, f, indent=2)
            
        print(f"\nProcessed {len(trajectories)} questions. Results saved to {output_file}")
        
    except Exception as e:
        print(f"Error generating dataset: {e}")

INPUT_FILE = "input.json"
OUTPUT_FILE = "text2sql_data_file_auto.json"

athena_client = boto3.client('athena')

generate_dataset(INPUT_FILE, OUTPUT_FILE,athena_client)



Processing question 0: What is the highest eligible free rate for K-12 students in the schools in Alameda County?
Generated ground truth for question 0

Processing question 1: Please list the lowest three eligible free rates for students aged 5-17 in continuation schools.
Generated ground truth for question 1

Processing question 2: Please list the zip code of all the charter schools in Fresno County Office of Education.
Generated ground truth for question 2

Processing question 3: What is the unabbreviated mailing street address of the school with the highest FRPM count for K-12 students?
Generated ground truth for question 3

Processing question 4: Please list the phone numbers of the direct charter-funded schools that are opened after 2000/1/1.
Generated ground truth for question 4

Processed 5 questions. Results saved to text2sql_data_file_auto.json


## Run Bedrock Agent Evaluation Framework on the newly created sample Text2SQL agent

Step 1: Navigate to config_tpl.py at the root of the repository and create a copy named 'config.py'

Step 2: Fill in config.py with the below information

In [57]:
print("AGENT_ID= \"{}\"".format(text2sql_agent_id))
print("AGENT_ALIAS_ID= \"{}\"".format(text2sql_agent_alias_id))
print("DATA_FILE_PATH= \"{}\"".format("blog_sample_agents/2-Sample-text2sql-agent/text2sql_data_file_auto.json"))

AGENT_ID= "T1GQGO9KJI"
AGENT_ALIAS_ID= "2ZLVI0FRSW"
DATA_FILE_PATH= "blog_sample_agents/2-Sample-text2sql-agent/text2sql_data_file_auto.json"


Step 3: Input the keys provided by your Langfuse project into the 'config.py' file

Step 4: Run the evaluation framework against the dataset by opening a terminal, navigate to the root of the repository, and run 'python3 driver.py' 